In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.features= nn.Sequential(
            nn.Conv2d(kernel_size=3,in_channels=3,out_channels=96,stride=4,padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3,stride=2),
        )


    def forward(self,X):


